![](https://i.imgur.com/qkg2E2D.png)

# UnSupervised Learning Methods

## Exercise 003 - Part II

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 23/05/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_03/Exercise0002Part002.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml, load_breast_cancer, load_digits, load_iris, load_wine

# Computer Vision

# Miscellaneous
import os
import math
from platform import python_version
import random
import time
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

DATA_FILE_URL   = r'None'
DATA_FILE_NAME  = r'None'

T_MNIST_IMG_SIZE = (28, 28)


In [ ]:
# Auxiliary Functions

def BalancedSubSample( dfX: pd.DataFrame, colName: str, numSamples: int ):
    
    # TODO: Validate the number of samples
    # TODO: Validate the column name (Existence and categorical values)
    return dfX.groupby(colName, as_index = False, group_keys = False).apply(lambda dfS: dfS.sample(numSamples, replace = False))

## Guidelines

 - Fill the full names and ID's of the team members in the `Team Members` section.
 - Answer all questions / tasks within the Jupyter Notebook.
 - Use MarkDown + MathJaX + Code to answer.
 - Verify the rendering on VS Code.
 - Submission in groups (Single submission per group).
 - You may and _should_ use the forums for questions.
 - Good Luck!

* <font color='brown'>(**#**)</font> The `Import Packages` section above imports most needed tools to apply the work. Please use it.
* <font color='brown'>(**#**)</font> You may replace the suggested functions to use with functions from other packages.
* <font color='brown'>(**#**)</font> Whatever not said explicitly to implement maybe used by a 3rd party packages.
* <font color='brown'>(**#**)</font> The total run time of this notebook must be **lower than 60 [Sec]**.

## Team Members
- Nadav_Talmon_203663950
- Nadav_Shaked_312494925
- Adi_Rosenthal_316550797

## Generate / Load Data

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if (DATA_FILE_NAME != 'None') and (not os.path.exists(DATA_FILE_NAME)):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

## 3. PCA

### 3.1. PCA Algorithm

In this section we'll implement a SciKit Learn API compatible class for the PCA.  
The class should implement the following methods:

1. `__init____()` - The object constructor by the encoder dimension.  
2. `fit()` - Given a data set builds the encoder / decoder.  
3. `transform()` - Applies the encoding on the input data.  
4. `inverse_transform()` - Applies the decoding on the input data.  

* <font color='brown'>(**#**)</font> You may use the [SciKit Learn's PCA module](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) as a reference.
* <font color='brown'>(**#**)</font> Both encoding and decoding applied as out of sample encoding / decoding.
* <font color='brown'>(**#**)</font> Pay attention to data structure (`N x D`).


In [ ]:
class PCA:
    def __init__(self, d: int = 2):
        '''
        Constructing the object.
        Args:
            d - Number of dimensions of the encoder output.
        '''
        # ===========================Fill This===========================#
        # 1. Keep the model parameters.

        self.d = d
        self.encoder = None
        self.decoder = None
        # ===============================================================#

    def fit(self, mX: np.ndarray):
        '''
        Fitting model parameters to the input.
        Args:
            mX - Input data with shape N x D.
        Output:
            self
        '''
        # ===========================Fill This===========================#
        # 1. Build the model encoder.
        # 2. Build the model decoder.
        # 3. Optimize calculation by the dimensions of `mX`.
        # !! You may find `scipy.sparse.linalg.svds()` useful.
        # !! You may find `scipy.sparse.linalg.eigsh()` useful.

        N, D = mX.shape

        vMean = np.mean(mX, axis=0).reshape(1, -1)
        mX_centers = mX - vMean

        if D <= N:
            eigenvalues, eigenvectors = sp.sparse.linalg.eigsh(mX_centers.T @ mX_centers, k=self.d)
            mUd = eigenvectors[:, np.argsort(eigenvalues)[::-1]]
        else:
            v_d, Σ_x_power2, _ = sp.sparse.linalg.svds(mX_centers @ mX_centers.T, k=self.d)
            eigenvectors = (mX_centers.T @ v_d @ (np.linalg.inv(np.sqrt(np.diag(Σ_x_power2)))))
            mUd = eigenvectors[:, np.argsort(Σ_x_power2)[::-1]]

        self.encode = lambda mX: (mX - vMean) @ mUd
        self.decode = lambda mZ: mZ @ mUd.T + vMean

        # ===============================================================#

        return self

    def transform(self, mX: np.ndarray) -> np.ndarray:
        '''
        Applies (Out of sample) encoding
        Args:
            mX - Input data with shape N x D.
        Output:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        '''
        # ===========================Fill This===========================#
        # 1. Encode data using the model encoder.

        mZ = self.encode(mX)

        # ===============================================================#

        return mZ

    def inverse_transform(self, mZ: np.ndarray) -> np.ndarray:
        '''
        Applies (Out of sample) decoding
        Args:
            mZ - Low dimensional representation (embeddings) with shape N x d.
        Output:
            mX - Reconstructed data with shape N x D.
        '''
        # ===========================Fill This===========================#
        # 1. Encode data using the model decoder.

        mX = self.decode(mZ)

        # ===============================================================#

        return mX

* <font color='red'>(**?**)</font> In the class we use _out of sample_ encoding / decoding. What if we use the same `mX` for training and the encoding?  
Make sure to understand this before proceeding.

### 3.2. PCA Application

In this section the PCA (Using the above class) will be applied on several data sets:

 * Breast Cancer Dataset - Loaded using `load_breast_cancer()`.
 * Digits Dataset - Loaded using `load_digits()`.
 * Iris Dataset - Loaded using `load_iris()`.
 * Wine Dataset - Loaded using `load_wine()`.

For each data set:

1. Make yourself familiar with the data set:
    * How many features are there ($D$).
    * How many samples are there ($D$).
    * Do all features have the same unit?
2. Apply a Pre Process Step  
   In ML, usually, if the features do not have the same unit they are normalized.  
   Namely, make each feature with zero mean and unit standard deviation.   
   Write a function to normalize input data.
3. Apply the PCA  
   Set `d` to be visualization friendly and apply PCA from $D$ to $d$.  
   The obtained the low dimensional data represents $\boldsymbol{Z} \in \mathbb{R}^{d \times N}$.
4. Plot Low Dimensional Data  
   Make a scatter plot of $\boldsymbol{Z} \in \mathbb{R}^{d \times N}$ and color the data points according to the data labels.  
   For each data set show result with the normalization step and without it.
5. Calculate Lost Energy  
   For each plot, show the value of ${\left\| \tilde{\boldsymbol{X}} - \boldsymbol{X} \right\|}_{F}^{2}$.  
   Do this by applying `inverse_transform()` on the low dimensional data and calculate the norm.


* <font color='brown'>(**#**)</font> Pay attention to the difference in dimensions of the data to the derived Math formulations.
* <font color='brown'>(**#**)</font> The output should be 2 figures for each data set. You may show them in a single plot using sub plots.

In [ ]:
def NormalizeData(mX: np.ndarray) -> np.ndarray:
    '''
    Normalize data so each feature has zero mean and unit standard deviation.
    Args:
        mX  - Input data with shape N x d.
    Output:
        mY  - Output data with shape N x d.
    Remarks:
        - Features with zero standard deviation are not scaled (Only centered).
    '''

    stds = np.std(mX, axis=0, ddof=0)
    stds[stds == 0] = 1
    mean = np.mean(mX, axis=0)
    mY = (mX - mean) / stds

    return mY

In [ ]:
#===========================Fill This===========================#
# 1. Set parameter `d`.
# 2. Load each data set.
# 3. Apply PCA to each data set with and without normalization.
# 4. Display results as scatter data.

def apply_pca(data, d, data_name):
    mX = data.data
    mX_normalized = NormalizeData(mX)

    pca_norm = PCA(d=d)
    pca_norm.fit(mX_normalized)
    mZ_norm = pca_norm.transform(mX_normalized)
    mX_reconstructed_norm = pca_norm.inverse_transform(mZ_norm)
    lost_energy_norm = np.linalg.norm(mX_reconstructed_norm - mX_normalized, ord='fro') ** 2

    pca_no_norm = PCA(d=d)
    pca_no_norm.fit(mX)
    mZ_no_norm = pca_no_norm.transform(mX)
    mX_reconstructed = pca_no_norm.inverse_transform(mZ_no_norm)
    lost_energy_no_norm = np.linalg.norm(mX_reconstructed - mX, ord='fro') ** 2

    # Plot the low-dimensional data
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
    fig.suptitle(f"dataset: {data_name}")

    # Without normalization
    axes[0].scatter(mZ_norm[:, 0], mZ_norm[:, 1], c=data.target)
    axes[0].set_title(f'With Normalization, Lost Energy: {lost_energy_norm}')
    axes[0].set_xlabel('PC1')
    axes[0].set_ylabel('PC2')

    # With normalization
    axes[1].scatter(mZ_no_norm[:, 0], mZ_no_norm[:, 1], c=data.target)
    axes[1].set_title(f'Without Normalization, Lost Energy: {lost_energy_no_norm}')
    axes[1].set_xlabel('PC1')
    axes[1].set_ylabel('PC2')

    plt.tight_layout()
    plt.show()


d = 2

breast_cancer = load_breast_cancer()
digits = load_digits()
iris = load_iris()
wine = load_wine()

# Apply PCA to each data set with and without normalization
apply_pca(breast_cancer, d, "breast_cancer")
apply_pca(digits, d, "digits")
apply_pca(iris, d, "iris")
apply_pca(wine, d, "wine")

#===============================================================#

### 3.3. Question

In the above, why does the results of the normalized and non normalized data are different?  
Address the geometry of the results and the value of the reconstruction error.

### 3.3. Solution

Normalization ensures that each feature contributes equally to PCA by equalizing their contributions through centering and scaling. This allows PCA to accurately identify the directions of maximum variance in the data.

When data is not normalized, features with larger variances can overshadow others, leading to distorted results and a biased representation of the data's geometry.

Additionally, normalization reduces the reconstruction error in PCA. Normalized data allows the PCA model to reconstruct the original data in a better way since it captures the dominant patterns more accurately. In contrast, non-normalized data with varying scales and means can result in higher reconstruction errors because the model may struggle to represent the original data structure effectively.


## 4. Image Denoising

In this section the PCA algorithm will be used for denoising images from the [MNIST Dataset](https://en.wikipedia.org/wiki/MNIST_database).  
In this section:

 1. Load Data  
    Load the MNIST data set and sub sample it.  
    We'll have a perfectly balanced data set.
    The data will be in `mX` and labels in `vY`.  
    This is already implemented.
 2. Add Noise  
    We'll add noise to the data.  
    The noise of the data will be modeled as a Poisson Noise (Also known as [_Shot Noise_](https://en.wikipedia.org/wiki/Shot_noise)).  
    The _Shot Noise_ is a classic model of noise gathered by imaging sensors.
 3. Analyze the Data  
    Analyze the spectrum of the data and choose an appropriate ste of parameters for denoising.
 3. Apply Denoising  
    Apply denoising by utilizing the PCA algorithm.
 4. Analyze Result  
    Show the results as a function of the parameters.

In [ ]:
# Parameters
numSamplesClass = 600
λ               = 35

In [ ]:
# Load Data
dfX, dfY = fetch_openml(name = 'mnist_784', version = 1, return_X_y = True, as_frame = True)#, parser = 'auto')

In [ ]:
# Sub Sample Data
dfData = pd.concat((dfX, dfY), axis = 1)

# Balanced Sub Sample
# End Result: 'numSamplesClass' samples per digit
dfData = BalancedSubSample(dfData, 'class', numSamplesClass)
vY = dfData['class'].to_numpy(dtype = np.uint8)
mX = dfData.drop(columns = ['class']).to_numpy()

In [ ]:
# Add Poisson Noise
mN = np.random.poisson(λ, size = mX.shape) #<! Noise samples

In [ ]:
# Add Noise
# Make sure values are in {0, 1, 2, ..., 255} range
mXRef = mX.copy() #<! Reference with no noise
mXRef = mXRef / 255

mX += mN
mX = np.minimum(mX, 255)


In [ ]:
# Show Samples

lIdx = [np.flatnonzero(vY == ii)[0] for ii in range(10)]

_, mHA = plt.subplots(1, 10, figsize = (16, 4))
for ii, hA in enumerate(mHA.flat):
    idx = lIdx[ii]
    mI  = np.reshape(mX[idx], T_MNIST_IMG_SIZE)
    # mI  = np.clip(mI, 0, 1)
    hA.imshow(mI, cmap = 'gray')
    hA.axis('off')
    
plt.tight_layout()
plt.show()

### 4.1. The Data Spectrum

In this section:

 1. Pre Process the data (Optional).  
    Do this step if oyu think it is needed.
 2. Plot the Spectrum of the Eigen Values of the data.
 3. Choose **a range** (5 values) of `d` for the low dimensionality reduction.
 4. For each `d` value, calculate the **relative energy loss**.


In [ ]:
#===========================Fill This===========================#
# 1. Pre Process Data (Optional).
# !! Make sure to keep the name of the data `mX`.
# !! Don't change the order of the data so it matches `vY`.

mX = mX / 255

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Calculate the spectrum of the Eigen Values of the data.

eigenvalues = np.linalg.eigvals(np.cov(mX.T))
eigenvalues_norm = eigenvalues / np.sum(eigenvalues)
sorted_eigenvalues = np.sort(eigenvalues)[::-1]
energy = [eigenvalues_norm[0]]
for e in range(len(eigenvalues_norm) - 1):
    energy.append(energy[-1] + eigenvalues_norm[e + 1])

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Display the Spectrum.
# !! You may show both the spectrum and the relative energy.

plt.figure(figsize=(8, 6))
plt.plot(sorted_eigenvalues, marker='o', linestyle='-', color='b', label="spectrum")
plt.xlabel('Eigenvalue Index')
plt.ylabel('Normalized Eigenvalue')
plt.title('Spectrum of Eigenvalues')
plt.grid(True)
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(energy, marker='+', linestyle='-', color='g', label="relative energy")
plt.xlabel('Eigenvalue Index')
plt.ylabel('relative energy')
plt.title('relative energy of Eigenvalues')
plt.grid(True)
plt.show()

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Choose a range of `d` values.
# 2. Per `d` plot / display the relative energy loss.
# !! Don't choose too many, keep running time and visualization reasonable.
# !! The choice should be in order to show the effect of `d` on the results and not only the optimal `d`.

d_values = [5, 20, 50, 100, 200, 400]
relative_energy_loss = []

for d in d_values:
    energy_loss = np.sum(sorted_eigenvalues[d:]) / np.sum(sorted_eigenvalues)
    relative_energy_loss.append(energy_loss)

# Display the relative energy loss
plt.figure(figsize=(8, 6))
plt.plot(d_values, relative_energy_loss, marker='o', linestyle='-', color='r')
plt.xlabel('d')
plt.ylabel('Relative Energy Loss')
plt.title('Relative Energy Loss for Different d Values')
plt.grid(True)
plt.show()

#===============================================================#

### 4.2. PCA Based Denoising

In this section, per `d` value:

 1. Build the _Encoder_ and _Decoder_. 
 2. Denoise the images listed in the index list `lIdx`.
 3. Show results per `d`
      * For each image show the reconstruction error vs. the noisy sample (`mX`).
      * For each image show the estimation error vs. the non noisy sample (`mXRef`).

* <font color='brown'>(**#**)</font> Make sure when you use the whole data (`mX`) and when the sub set to analyze.
* <font color='brown'>(**#**)</font> For the PCA you may only use `mX`.
* <font color='brown'>(**#**)</font> The output should be the 10 images per row where the number of rows is the number of `d` values + 2 (For the reference / noisy images).

In [ ]:
#===========================Fill This===========================#
# 1. Build the encoder / decoder using the `PCA` class above.
# 2. Per `d` denoise the images in `lIdx`.
# !! Only use `mX` for the PCA step.

denoised_images = []
reconstruction_errors = []
estimation_errors = []

for d in d_values:
    pca = PCA(d=d)
    pca.fit(mX)
    mX_lIdx = mX[lIdx]
    mZ = pca.transform(mX_lIdx)
    mX_reconstructed = pca.inverse_transform(mZ)
    denoised_images.append(mX_reconstructed.copy())
    reconstruction_error = np.linalg.norm(mX_reconstructed - mX[lIdx],axis=1)
    estimation_error = np.linalg.norm(mX_reconstructed -mXRef[lIdx],axis=1)
    reconstruction_errors.append(reconstruction_error.copy())
    estimation_errors.append(estimation_error.copy())

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Create a subplot of `len(d) + 2 x 10` plots.
# 2. In the 1st row, show the clean images (`mXRef`).
# 3. In the 2nd row, show the noisy images (`mX`).
# 4. In the next rows show the sample per different `d`.  
#    Per row, show `d`.

num_images = len(lIdx)
num_rows = len(d_values) + 2

fig = plt.figure(tight_layout=True, figsize=(20, 20))
gs = gridspec.GridSpec(num_rows, num_images)

ax = fig.add_subplot(gs[0, :])
ax.set_title(f'clean images (`mXRef`)')
ax.axis('off')

ax = fig.add_subplot(gs[1, :])
ax.set_title(f'noisy images (`mX`)')
ax.axis('off')

for i in range(len(d_values)):
    ax = fig.add_subplot(gs[i + 2, :])
    ax.set_title(f'd = {d_values[i]}')
    ax.axis('off')

for i, idx in enumerate(lIdx):
    ax = fig.add_subplot(gs[0, i])
    ax.imshow(mXRef[idx].reshape(T_MNIST_IMG_SIZE), cmap='gray')
    ax.axis('off')

for i, idx in enumerate(lIdx):
    ax = fig.add_subplot(gs[1, i])
    ax.imshow(mX[idx].reshape(T_MNIST_IMG_SIZE), cmap='gray')
    ax.axis('off')

# Plot denoised images for each d
for row in range(2, num_rows):
    for i, idx in enumerate(lIdx):
        d = d_values[row - 2]
        ax = fig.add_subplot(gs[row, i])
        ax.imshow(denoised_images[row - 2][i].reshape(T_MNIST_IMG_SIZE), cmap='gray')
        ax.axis('off')

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Create 2 sub plots where the `x` is the image index {0, 1, ..., 9}.
# 2. The 1st plot, per `d`, shows the reconstruction error.
# 3. The 2nd plot, per `d`, shows the estimation error.

fig, axs = plt.subplots(2, 1, sharex=True, sharey=True,figsize=(10, 10))
bar_width = 0.15
bar_positions = np.arange(num_images)
for i in range(len(d_values)):
    axs[0].bar(bar_positions + i * bar_width, reconstruction_errors[i], bar_width, label=f'd={d_values[i]}')

for i in range(len(d_values)):
    axs[1].bar(bar_positions + i * bar_width, estimation_errors[i], bar_width, label=f'd={d_values[i]}')

axs[1].set_xlabel('Image Index')
axs[0].set_ylabel('Reconstruction Error')
axs[1].set_ylabel('Estimation Error')

axs[0].legend(title='d values',loc='right',  bbox_to_anchor=(1.15, 1))

fig.subplots_adjust(hspace=0.4)
plt.show()

#===============================================================#

### 4.3. Question

Address the following remarks:

 - How does the noise model effect the performance of the denoising?  
   Specifically, if the noise model was Gaussian with the same variance, what would change?
 - Would you use the reconstruction error as an estimation of the estimation error?
 - Explain the idea behind the PCA denoising.  
   Specifically address the trade off between small and large values of `d`.
 - If the data was 1D, would you expect it to perform better?  
   Think if the model has any knowledge about the data being 2D.

### 4.3. Solution

The noise model can affect the performance of denoising. If the noise model was Gaussian with the same variance, the performance of denoising using PCA would still be affected. However, PCA assumes that the noise is additive and uncorrelated, which is similar to the assumptions of Gaussian noise. Therefore, the performance may still be reasonably good, but it would depend on the specific characteristics of the data and noise.

The reconstruction error cannot be used as an estimation of the estimation error. The reconstruction error measures the dissimilarity between the original image and the denoised image, while the estimation error measures the dissimilarity between the denoised image and the ground truth (non-noisy) image. The reconstruction error does not account for the noise in the original image, so it cannot accurately estimate the performance of denoising.

The idea behind PCA denoising is to project the noisy data onto a lower-dimensional subspace that captures the most important variations in the data. By reducing the dimensionality, PCA can effectively denoise the data by removing the noise components in the higher-dimensional space. The trade-off between small and large values of d (the number of principal components) lies in the balance between noise removal and preserving important information. Small values of d may not capture enough variations in the data, leading to incomplete denoising and loss of important details. On the other hand, large values of d may capture more noise components and lead to overfitting, resulting in less effective denoising. The optimal value of d depends on the specific characteristics of the data and the noise.

If the data was 1D, the performance of PCA denoising would likely be similar or even better compared to 2D data. PCA does not require knowledge about the data being 2D. It is a dimensionality reduction technique that can be applied to data of any dimensionality. However, in the case of 2D data, such as images, PCA can capture both row-wise and column-wise variations, which can be beneficial for denoising. In 1D data, PCA would capture variations along a single dimension, which may still be effective for denoising, especially if the noise is additive and uncorrelated.

> Royi: In `4.1.` Think about the geometry of the data.  
> Royi: In `4.2.` The idea was about measureing the error for images. Reconstruction is ${L}_{2}$ error.  
> Royi: In `4.4.` On the contrary.  
> Royi: ❌, See notes above.  
> Royi: <font color='red'>-2</font>.

### 4.4. PCA Denoising with Labels

In the above we used no knowledge on the label of the image.  
In this section you should use the labels information in order to improve results.

 1. Write a code which take advantage of the labels `vY` (Be creative).
 2. Show the plots of the reconstruction and estimation error.
 3. Explain, in words, your idea.
 4. Explain, in words, the results.

In [ ]:
#===========================Fill This===========================#
# 1. Choose the maximum `d` used in the previous section.
# 2. Apply PCA Denoising on the list of images.

reconstruction_errors_with_labels = []
estimation_errors_with_labels = []

labels = range(10)
d = d_values[-1]

for label in labels:
    pca = PCA(d=d)
    pca.fit(mX[vY == label])
    mX_lIdx = mX[lIdx[label]]
    mZ = pca.transform(mX_lIdx)
    mX_reconstructed = pca.inverse_transform(mZ)

    reconstruction_error = np.linalg.norm(mX_reconstructed - mX[lIdx[label]])
    estimation_error = np.linalg.norm(mX_reconstructed - mXRef[lIdx[label]])
    reconstruction_errors_with_labels.append(reconstruction_error.copy())
    estimation_errors_with_labels.append(estimation_error.copy())

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Display the reconstruction and estimation error per image.
# 2. Compare to the previous result for the same `d`.
reconstruction_errors_no_label = reconstruction_errors[-1]
estimation_errors_no_label = estimation_errors[-1]

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(10, 10))
fig.suptitle(f"reconstruction and estimation error for d = {d}")
bar_width = 0.15
bar_positions = np.arange(num_images)

axs[0].bar(bar_positions, reconstruction_errors_no_label, bar_width, label=f'Without label')
axs[0].bar(bar_positions + bar_width, reconstruction_errors_with_labels, bar_width, label=f'With label')

axs[1].bar(bar_positions, estimation_errors_no_label, bar_width, label=f'Without label')
axs[1].bar(bar_positions + bar_width, estimation_errors_with_labels, bar_width, label=f'With label')

axs[1].set_xlabel('Image Index')
axs[0].set_ylabel('Reconstruction Error')
axs[1].set_ylabel('Estimation Error')

axs[0].legend(title='With \ Without label',loc='upper right')
axs[1].legend(title='With \ Without label',loc='upper right')

#===============================================================#

### 4.4.3. Solution

The idea is to utilize the labels vY associated with the images to enhance the denoising process by running PCA for each label seperatly. 
By considering the labels associated with the images, we can take advantage of label-dependent characteristics that may exist in the data. These characteristics can capture specific patterns or variations present in images of different classes.
By incorporating this information into the denoising process, we can potentially enhance the reconstruction and estimation of image features that are relevant to each class.


### 4.4.4. Solution

In our results, we observed that for high-dimensional data (in our case, d=400), the estimation error remained the same while the reconstruction error decreased as we incorporated label information into the denoising process. This means that considering the labels associated with the images helped improve the accuracy of reconstructing the original image while maintaining the same level of estimation error.